In [ ]:
#Вашим заданием в этом модуле будет проанализировать и преобразовать данные о велопоездках клиентов компании Citi Bike (США), 
# специализирующейся на прокате велосипедов.
#Датасет представляет собой таблицу с информацией о 300 тысячах поездок за первые пять дней сентября 2018 года и включает в себя следующую информацию:

#starttime — время начала поездки (дата, время);
#stoptime — время окончания поездки (дата, время);
#start station id — идентификатор стартовой стоянки;
#start station name — название стартовой стоянки;
#start station latitude, start station longitude — географическая широта и долгота стартовой стоянки;
#end station id — идентификатор конечной стоянки;
#end station name — название конечной стоянки;
#end station latitude, end station longitude — географическая широта и долгота конечной стоянки;
#bikeid — идентификатор велосипеда;
#usertype — тип пользователя (Customer — клиент с подпиской на 24 часа или на три дня, Subscriber — подписчик с годовой арендой велосипеда);
#birth year — год рождения клиента;
#gender — пол клиента (0 — неизвестный, 1 — мужчина, 2 — женщина).

In [ ]:
import pandas as pd

bike_data = pd.read_csv('data/citibike-tripdata.csv', sep=',')
bike_data.head()

In [2]:
bike_df = bike_data.copy()

In [ ]:
bike_df.info()

In [ ]:
#Найдите идентификатор самой популярной стартовой стоянки. Запишите идентификатор в виде целого числа.
bike_df['start station id'].mode()

In [ ]:
#Велосипед с каким идентификатором является самым популярным?
bike_df['bikeid'].mode()[0]

In [ ]:
#Какой тип клиентов (столбец usertype) является преобладающим — Subscriber или Customer? 
# В качестве ответа запишите долю клиентов преобладающего типа среди общего количества клиентов. Ответ округлите до сотых.
bike_df['usertype'].value_counts(normalize=True)

In [ ]:
# Кто больше занимается велоспортом — мужчины или женщины? В ответ запишите число поездок для той группы, у которой их больше.
bike_df['gender'].value_counts()

In [ ]:
#
bike_df['start station id'].count
bike_df['end station id'].count

In [ ]:
bike_df['birth year'].max()

In [ ]:
bike_df['start station name'].value_counts()

In [ ]:
bike_df['end station name'].value_counts()

In [ ]:
#В первую очередь удалим лишнюю информацию из данных.
#В наших данных присутствуют столбцы, которые дублируют информацию друг о друге: это столбцы с идентификатором и названием стартовой и конечной стоянки. 
# Удалите признаки идентификаторов стоянок. Сколько столбцов осталось?
bike_df.drop(['start station id','end station id'], axis=1)

In [ ]:
#Замените признак birth year на более понятный признак возраста клиента age. Годом отсчёта возраста выберите 2018 год. 
# Столбец birth year удалите из таблицы. 
bike_df['age'] = 2018 - bike_df['birth year']
bike_df.drop(['birth year'], axis=1)

In [ ]:
#Сколько поездок совершено клиентами старше 60 лет?
bike_df[bike_df['age'] > 60].shape[0]

In [ ]:
 #   Создайте признак длительности поездки trip duration. 
 # Для этого вычислите интервал времени между временем окончания поездки (stoptime) и временем её начала (starttime) в секундах. 
 # В качестве ответа запишите среднюю длительность поездки в секундах. Ответ округлите до целого.
display(bike_df['starttime'])
bike_df['starttime'] = pd.to_datetime(bike_df['starttime'])
bike_df['stoptime'] = pd.to_datetime(bike_df['stoptime'])

display(bike_df['starttime'])

In [ ]:
#import datetime #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
bike_df['trip duration'] = bike_df['stoptime'] - bike_df['starttime']

def time_to_sec(time_std):
    n_seconds = time_std.dt.days*86400 + time_std.dt.hours*3600 + time_std.dt.minutes*60 + time_std.dt.seconds
    return n_seconds

trip_duration = bike_df['trip duration']#.apply(time_to_sec)

display(trip_duration)
display(trip_duration.dt.seconds.mean())
    

In [ ]:
#!!!! Accessor .dt onfy for pandas.Series but not for datetime type!!!!
display(bike_df['starttime'])
bike_df['weekend'] = bike_df['starttime'].apply(lambda x: 1 if x.dayofweek >=5 else 0)
display(bike_df['weekend'].value_counts())

In [ ]:
#Создайте признак времени суток поездки time_of_day. Время суток будем определять из часа начала поездки. Условимся, что:
#поездка совершается ночью (night), если её час приходится на интервал от 0 (включительно) до 6 (включительно) часов;
#поездка совершается утром (morning), если её час приходится на интервал от 6 (не включительно) до 12 (включительно) часов;
#поездка совершается днём (day), если её час приходится на интервал от 12 (не включительно) до 18 (включительно) часов;
#поездка совершается вечером (evening), если её час приходится на интервал от 18 (не включительно) до 23 часов (включительно).
#Во сколько раз количество поездок, совершённых днём, больше, чем количество поездок, совёршенных ночью, за представленный в данных период времени? 
# Ответ округлите до целых.

def get_part_of_day(t_time):
    t_hour = t_time.hour
    if t_hour >= 0 and t_hour <= 6:
        return 'night'
    if t_hour > 6 and t_hour <= 12:
        return 'morning'
    if t_hour > 12 and t_hour <= 18:
        return 'day'
    if t_hour > 18 and t_hour <= 23:
        return 'evening'


bike_df['time_of_day'] = bike_df['starttime'].apply(get_part_of_day)
display(bike_df['time_of_day'])
display(bike_df['time_of_day'].value_counts())

In [33]:
display(bike_df['time_of_day'].value_counts()['day']/bike_df['time_of_day'].value_counts()['night'])

9.480411004308916